In [6]:
!pip install tokenizers

     |█████████▌                      | 1.1 MB 49 kB/s eta 0:00:520ERROR: Exception:
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.9/site-packages/pip/_vendor/urllib3/response.py", line 438, in _error_catcher
    yield
  File "/opt/anaconda3/lib/python3.9/site-packages/pip/_vendor/urllib3/response.py", line 519, in read
    data = self._fp.read(amt) if not fp_closed else b""
  File "/opt/anaconda3/lib/python3.9/site-packages/pip/_vendor/cachecontrol/filewrapper.py", line 62, in read
    data = self.__fp.read(amt)
  File "/opt/anaconda3/lib/python3.9/http/client.py", line 463, in read
    n = self.readinto(b)
  File "/opt/anaconda3/lib/python3.9/http/client.py", line 507, in readinto
    n = self.fp.readinto(b)
  File "/opt/anaconda3/lib/python3.9/socket.py", line 704, in readinto
    return self._sock.recv_into(b)
  File "/opt/anaconda3/lib/python3.9/ssl.py", line 1241, in recv_into
    return self.read(nbytes, buffer)
  File "/opt/anaconda3/lib/python3.9/ssl.py

In [ ]:
!pip install transformers

In [ ]:
!pip install pytorch
!pip install tqdm

In [1]:
from dask.distributed import Client
# when working with clusters, specify cluster config, n_workers and worker_size
client = Client(n_workers=4, 
                       threads_per_worker=1,
                       memory_limit=0)

2022-07-15 18:30:52,604 - distributed.diskutils - INFO - Found stale lock file and directory '/Users/mika.kimmins/projects/scalingpythonml/dask/dask-worker-space/worker-uq3h8rw9', purging
2022-07-15 18:30:52,605 - distributed.diskutils - INFO - Found stale lock file and directory '/Users/mika.kimmins/projects/scalingpythonml/dask/dask-worker-space/worker-u47sujg_', purging


In [2]:
import dask.dataframe as dd
import pandas as pd


In [ ]:
# https://github.com/facebookresearch/fairseq/blob/main/examples/translation/README.md

In [ ]:
# https://www.kaggle.com/code/abhishek/roberta-inference-5-folds

In [3]:
import numpy as np
import pandas as pd
import os
import tokenizers
import string
import torch
import transformers
import torch.nn as nn
from torch.nn import functional as F
from tqdm import tqdm
import re

ModuleNotFoundError: No module named 'tokenizers'

In [ ]:
MAX_LEN = 192
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 8
EPOCHS = 1
# ROBERTA_PATH = "../input/roberta-base"
# TOKENIZER = tokenizers.ByteLevelBPETokenizer(
#     vocab_file=f"{ROBERTA_PATH}/vocab.json", 
#     merges_file=f"{ROBERTA_PATH}/merges.txt", 
#     lowercase=True,
#     add_prefix_space=True
# )

In [ ]:
# https://huggingface.co/roberta-base
from transformers import RobertaTokenizer, RobertaModel
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaModel.from_pretrained('roberta-base')
text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)

In [ ]:
# https://www.kaggle.com/code/arvissu/roberta-base-inference-v2-0

In [ ]:
import gc
import os
import sys
import time
import pickle
import random
from tqdm import tqdm
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold

import torch
import transformers
import torch.nn as nn
import torch.nn.functional as F
from torch.cuda.amp import GradScaler, autocast
from torch.utils.data import Dataset, DataLoader
from transformers import AutoModel, AutoTokenizer, AdamW, get_linear_schedule_with_warmup

import warnings
warnings.simplefilter('ignore')

In [ ]:
# download the model
# download the datasets from kaggle

In [7]:
def fetchEssay(essay_id: str):
    """
    Read the text file of the specific essay_id
    """
    essay_path = os.path.join('../input/feedback-prize-effectiveness/train/', essay_id + '.txt')
    essay_text = open(essay_path, 'r').read()
    return essay_text

# maybe?
def fetchEssay_ddf(df, essay_id: str):
    """
    Read the text file of the specific essay_id
    """
    essay_path = os.path.join('../input/feedback-prize-effectiveness/train/', essay_id + '.txt')
    essay_text = open(essay_path, 'r').read()
    return essay_text

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

In [ ]:
EPOCHS = 1
FOLDS = 4
lr = 2e-5
SEED = 2018
MAX_LEN = 512
BATCH_SIZE = 8
accumulation_steps = 4
seed_everything(SEED)

In [ ]:
class callback:
    def __init__(self):
        self.loss = list()
        self.model = list()
    
    def put(self, model, loss):
        self.loss.append(loss)
        self.model.append(model)

    def get_model(self):
        ind = np.argmin(self.loss)
        return self.model[ind]

    
class FeedBackModel(nn.Module):
    def __init__(self, model_path):
        super(FeedBackModel, self).__init__()
        self.model = AutoModel.from_pretrained(model_path)
        self.linear = nn.Linear(768, 3)

    def forward(self, ids, mask):
        x = self.model(ids, mask)[0][:, 0, :]
        pred = self.linear(x)
        return pred


class FeedBackDataset(Dataset):
    def __init__(self, data, model_path, is_test=False):
        self.data = data
        self.is_test = is_test
        self.tokenizer = AutoTokenizer.from_pretrained(model_path)
        
    def __getitem__(self, idx):
        text = self.data['discourse_text'].values[idx] + ' ' + self.tokenizer.sep_token*2  + ' '  + self.data['essay'].values[idx]
        if not self.is_test:
            target_value = self.data[y_cols].values[idx]
      
        inputs = self.tokenizer.encode_plus(
            text,
            None,
            truncation=True,
            add_special_tokens=True,
            max_length=MAX_LEN
        )['input_ids'] 
        
                  
        mask = [1]*len(inputs) + [0] * (MAX_LEN - len(inputs)) 
        mask = torch.tensor(mask, dtype=torch.long)
        
        if len(inputs) != MAX_LEN:
            inputs = inputs + [self.tokenizer.pad_token_id] * (MAX_LEN - len(inputs)) 
        ids = torch.tensor(inputs, dtype=torch.long)
        
        
        
        
        if self.is_test:
            return {
                'ids': ids,
                'mask': mask,
            }
        
        else:
            targets = torch.FloatTensor(target_value)
            return {
                'ids': ids,
                'mask': mask,
                'targets': targets
            }
        
    def __len__(self):
        return len(self.data)

In [ ]:
def load_model(model_path):
    """Return the model and tokenizer"""ƒ
    detoke = RobertaTokenizer.from_pretrained(model_path)
    model = RobertaForSequenceClassification.from_pretrained(model_path, return_dict=True)

    return detoke, model

In [ ]:
model_path = '../input/roberta-base/'
tokenizer, model_loaded = load_model(model_path)
dmodel = dask.delayed(model_loaded.cpu())

In [8]:
y_cols = ['discourse_effectiveness']

In [ ]:
# dask portion
src = "../input/feedback-prize-effectiveness/train.csv"
ddf = dd.read_csv(src)

ddf['essay'] = ddf['essay_id'].map_partitions(
    fetchessay
)

new_label = {"Ineffective": 0, "Adequate": 1, "Effective": 2}
ddf['discourse_effectiveness']  = ddf['discourse_effectiveness'].map_partitions(
   lambda x: new_label[x] 
)

In [ ]:
# https://www.kaggle.com/code/arvissu/roberta-base-training-notebook-1-epoch

In [ ]:
model_list = list()
kf = StratifiedKFold(n_splits=FOLDS)
for i, (train_idx, valid_idx) in enumerate(kf.split(df, y=df['essay_id'])):
    print(f'fold {i+1}')
    gc.collect()
    
    cb = callback()
    train_loader = torch.utils.data.DataLoader(FeedBackDataset(df.loc[train_idx, :].reset_index(drop=True), model_path), batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
    val_loader = torch.utils.data.DataLoader(FeedBackDataset(df.loc[valid_idx, :].reset_index(drop=True), model_path), batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
    
    
    net = FeedBackModel(model_path)
    net.cuda()
    
    loss_fn = torch.nn.CrossEntropyLoss()
    optimizer = AdamW(net.parameters(), lr = lr)    
    param_optimizer = list(net.named_parameters())
    no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]
    num_train_optimization_steps = int(EPOCHS * len(train_loader) / accumulation_steps)
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0.05 * num_train_optimization_steps,
                                                num_training_steps=num_train_optimization_steps)  # PyTorch scheduler
    scaler = torch.cuda.amp.GradScaler()
    
    for epoch in range(EPOCHS):  

        start_time = time.time()
        avg_loss = 0.0
        net.train()
        tbar = tqdm(train_loader, file=sys.stdout)
        loss_list = []
        val_loss_list = []
        
        for step, data in enumerate(tbar):

            # get the inputs
            input_ids = data['ids'].cuda()
            input_masks = data['mask'].cuda()
            targets = data['targets'].long().view(-1).cuda()
            with torch.cuda.amp.autocast():
                pred = net(input_ids,input_masks)
                loss = loss_fn(pred, targets)
                
            scaler.scale(loss).backward()
            

            if step % accumulation_steps == 0 or step == len(tbar) - 1:
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad()
                scheduler.step()
                
            loss_list.append(loss.detach().cpu().item())
            avg_loss = np.round(np.mean(loss_list), 4)

            tbar.set_description(f"Epoch {epoch + 1} Loss: {avg_loss} lr: {scheduler.get_last_lr()}")
        
        net.eval()
        avg_val_loss = 0.0   
        tbar_val = tqdm(val_loader, file=sys.stdout)
        for step, data in enumerate(tbar_val):

            # get the inputs
            input_ids = data['ids'].cuda()
            input_masks = data['mask'].cuda()
            targets = data['targets'].long().view(-1).cuda()
            
            pred = net(input_ids,input_masks)
            loss = loss_fn(pred, targets)
                
            val_loss_list.append(loss.detach().cpu().item())
            avg_val_loss = np.round(np.mean(val_loss_list), 4)

            tbar_val.set_description(f"Epoch {epoch + 1} Loss: {avg_val_loss}")
                    
        cb.put(net, avg_val_loss)   
        
    model_list.append(cb.get_model())

In [ ]:
# Done generating a model.
model_target = "roberta_modellist.pkl"
with open(model_target,"wb") as f:
    pickle.dump(model_list, f)


In [ ]:
# Infer

In [9]:
NFOLDS = 5
BATCH_SIZE = 32
MAX_LEN = 512


model_list = pickle.load(open(model_target, "rb"))
test_pred = np.zeros((len(test_df), 3))

# test_df = pd.read_csv("../input/feedback-prize-effectiveness/test.csv")
test_ddf = dd.read_csv("../input/feedback-prize-effectiveness/test.csv")

NameError: name 'torch' is not defined

In [ ]:

def predict(test_df: pd.DataFrame, tokenizer, result):
    test_loader = torch.utils.data.DataLoader(FeedBackDataset(test_df, tokenizer, True), batch_size=BATCH_SIZE, shuffle=False)
    with torch.no_grad():
        for i, data in enumerate(test_loader):
            input_ids = data['ids'].cuda()
            input_masks = data['mask'].cuda()
            pred = F.softmax(net(input_ids,input_masks))
            result.extend(pred.cpu().detach().numpy())

In [ ]:
for idx in range(NFOLDS):
    print(f'start to inference fold : {idx}')
    net = model_list[idx]
    net.eval()
    net.cuda()
    result = list()
    predict(test_ddf, tokenizer, result)
    test_pred += np.array(result)/NFOLDS
    

In [ ]:
# for idx in range(NFOLDS):
#     print(f'start to inference fold : {idx}')
#     net = model_list[idx]
#     net.eval()
#     net.cuda()
#     result = list()
#     with torch.no_grad():
#         for i, data in enumerate(test_loader):
#             input_ids = data['ids'].cuda()
#             input_masks = data['mask'].cuda()
#             pred = F.softmax(net(input_ids,input_masks))
#             result.extend(pred.cpu().detach().numpy())
#     test_pred += np.array(result)/NFOLDS

In [ ]:
# this is our test set predictions
test_pred
